<a href="https://colab.research.google.com/github/DavoII/handwritten_letters_recognition/blob/master/ARMENIAN_LETTERS_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [3]:
data = np.load('drive/My Drive/lowercase_agg_first_10/data.npy', encoding='bytes') 
labels = np.load('drive/My Drive/lowercase_agg_first_10/labels.npy', encoding='bytes') 
class_idx = np.load('drive/My Drive/lowercase_agg_first_10/label_classes.npy', encoding='bytes')

print(data.shape)
print(labels.shape)

(34512, 64, 64)
(34512, 10)


In [4]:
number_classes = len(class_idx)
print('number of classes', number_classes)

number of classes 10


In [0]:
x_train, x_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=labels
    )

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [0]:
(img_shape_w, img_shape_h) = x_train[0].shape
x_train = x_train.reshape(x_train.shape[0], img_shape_w, img_shape_h, 1)
x_test = x_test.reshape(x_test.shape[0], img_shape_w, img_shape_h, 1)

In [7]:
print('x_train', x_train.shape)
print('x_test', x_test.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)

x_train (24158, 64, 64, 1)
x_test (10354, 64, 64, 1)
y_train (24158, 10)
y_test (10354, 10)


In [0]:
ly1_neurons = 128
ly2_neurons = 256
ly3_neurons = 518

kernel_size_1 = (5, 5)
kernel_size_2 = (5, 5)
kernel_size_3 = (3, 3)

dropout_1 = 0.1
dropout_2 = 0.2
dropout_3 = 0.5

pool_size_1 = (2, 2)
pool_size_2 = (2, 2)
pool_size_3 = (2, 2)

categories=number_classes

model = Sequential()

model.add(Conv2D(ly1_neurons, kernel_size_1, input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=pool_size_1))
model.add(Dropout(dropout_1))

model.add(Conv2D(ly2_neurons, kernel_size_2))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=pool_size_2))
model.add(Dropout(dropout_2))

model.add(Conv2D(ly3_neurons, kernel_size_3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=pool_size_3))
model.add(Dropout(dropout_3))

model.add(Flatten())
model.add(Dense( 512 ))
model.add(Activation('relu'))
model.add(Dense( categories ))
model.add(Activation('softmax'))

In [0]:
learning_rate = 0.0001
model.compile(optimizer=optimizers.Adam(
    learning_rate=learning_rate
), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 128)       3328      
_________________________________________________________________
activation (Activation)      (None, 60, 60, 128)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 256)       819456    
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0

In [11]:
model.fit(
    x_train,
    y_train,
    batch_size=120,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
    )

Epoch 1/10
202/202 [==============================] - 33s 165ms/step - loss: 1.1821 - accuracy: 0.7008 - val_loss: 0.8785 - val_accuracy: 0.7489
Epoch 2/10
202/202 [==============================] - 33s 161ms/step - loss: 0.7118 - accuracy: 0.7831 - val_loss: 0.6631 - val_accuracy: 0.7966
Epoch 3/10
202/202 [==============================] - 33s 162ms/step - loss: 0.5119 - accuracy: 0.8408 - val_loss: 0.6981 - val_accuracy: 0.7644
Epoch 4/10
202/202 [==============================] - 33s 162ms/step - loss: 0.4186 - accuracy: 0.8689 - val_loss: 0.4596 - val_accuracy: 0.8524
Epoch 5/10
202/202 [==============================] - 33s 162ms/step - loss: 0.3622 - accuracy: 0.8861 - val_loss: 0.3403 - val_accuracy: 0.8918
Epoch 6/10
202/202 [==============================] - 33s 162ms/step - loss: 0.3076 - accuracy: 0.9024 - val_loss: 0.2690 - val_accuracy: 0.9171
Epoch 7/10
202/202 [==============================] - 33s 162ms/step - loss: 0.2609 - accuracy: 0.9161 - val_loss: 0.2316 - val_ac